## Notebook principal

Primero se busca trabajar con 3 libros de texto en formato PDF, los cuales son:  
Codigo limpio - Robert Cecil Martin  
Ingenieria de software - Pressman  
Ingenieria de software - Sommerville

In [1]:
# PyPDF2 es utilizado para leer cada pdf y extraer el texto de cada pagina.
%pip install PyPDF2
%pip install tqdm

  Using cached pypdf2-3.0.1-py3-none-any.whl (232 kB)
Note: you may need to restart the kernel to use updated packages.
  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
from PyPDF2 import PdfReader
from tqdm.auto import tqdm
from os import makedirs, listdir, path, getcwd

# get the current directory path of this jupyter notebook
BASE_DIR = getcwd()
DATA_DIR = path.join(BASE_DIR, "data")
PDF_DIR = path.join(DATA_DIR, "pdf")
TXT_DIR = path.join(DATA_DIR, "txt")

Una vez teniendo los libros en formato PDF, se procede a convertirlos a formato TXT, para poder trabajar con ellos.
La conversion ocurre pagina por pagina.

In [8]:
makedirs(TXT_DIR, exist_ok=True)
# obtener todos los archivos pdf dentro de PDF_DIR
pdf_files = [f for f in listdir(PDF_DIR) if f.lower().endswith('.pdf')]
for pdf in pdf_files:
    # leer el archivo pdf
    with open(path.join(PDF_DIR, pdf), 'rb') as f:
        pdf_reader = PdfReader(f)
        # leer el número de páginas en el archivo pdf
        pbar = tqdm(range(len(pdf_reader.pages)))
        # iterar sobre las páginas en el archivo pdf
        page_num = 0
        for page in pbar:
            pbar.set_description(f'Processing {pdf}')
            # obtener el texto de la página eliminando los saltos de línea
            text = pdf_reader.pages[page].extract_text().replace('\n', ' ')
            # escribir el texto en un archivo txt
            txt_file = f'{pdf[:-4]}_{page_num}.txt'
            # generar un archivo txt que contenga el texto de todos los archivos txt de un mismo pdf
            main_txt_file = f'{pdf[:-4]}.txt'
            with open(path.join(TXT_DIR, txt_file), 'w', encoding='utf-8') as f, open(path.join(TXT_DIR, main_txt_file), 'a', encoding='utf-8') as mwf:
                # formatear el texto para normalizarlo
                formatted_text = text.replace('\n', ' ').lower().replace('  ', ' ')
                f.write(formatted_text)
                mwf.write(formatted_text)
                page_num += 1

Processing codigo_limpio_robert_cecil_martin.pdf: 100%|██████████| 644/644 [00:13<00:00, 48.33it/s] 
Processing ingenieria_de_software_sommerville.PDF: 100%|██████████| 792/792 [00:04<00:00, 163.19it/s]
Processing ingenieria_de_software_pressman.PDF: 100%|██████████| 810/810 [00:09<00:00, 86.25it/s] 


Una vez teniendo todos los archivos .txt se debe obtener el dataset de STS lleno de oraciones relacionadas a la ingenieria de software y un puntaje entre 0 y 1 que valore la similitud entre las oraciones.

Se buscara hacer uso de algun modelo para extraer las oraciones del archivo .txt

In [10]:
%pip install huggingface_hub transformers datasets  

  Using cached huggingface_hub-0.15.1-py3-none-any.whl (236 kB)
  Using cached transformers-4.30.2-py3-none-any.whl (7.2 MB)
  Using cached datasets-2.13.1-py3-none-any.whl (486 kB)
  Using cached filelock-3.12.2-py3-none-any.whl (10 kB)
  Using cached fsspec-2023.6.0-py3-none-any.whl (163 kB)
  Using cached requests-2.31.0-py3-none-any.whl (62 kB)
  Using cached PyYAML-6.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (682 kB)
  Using cached typing_extensions-4.6.3-py3-none-any.whl (31 kB)
  Using cached numpy-1.25.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.6 MB)
  Using cached regex-2023.6.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (770 kB)
  Using cached tokenizers-0.13.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)
  Using cached safetensors-0.3.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
  Using cached pyarrow-12.0.1-cp310-cp310-manylinux_2_1

Buscando los modelos en espanol para extraer automaticamente las oraciones de ingenieria de software encontre este modelo: [hiiamsid/sentence_similarity_spanish_es](https://huggingface.co/hiiamsid/sentence_similarity_spanish_es) el cual potencialmente sera usado para realizarle fine-tuning y asi obtener un modelo final.